# 0. Install Dependencies and Bring in Data

In [ ]:
!pip install tensorflow tensorflow-gpu pandas matplotlib sklearn

In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import pickle

In [2]:
df = pd.read_csv(os.path.join('toxic_comments','train.csv'))

In [3]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


# 1. Preprocess

In [ ]:
!pip list

In [4]:
from tensorflow.keras.layers import TextVectorization

In [5]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [6]:
MAX_FEATURES = 200000 # number of words in the vocab

In [7]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

In [8]:
vectorizer.adapt(X.values)

In [ ]:
vectorizer.get_vocabulary()

In [9]:
vectorized_text = vectorizer(X.values)

In [10]:
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps bottlenecks

In [11]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

# 2. Create Sequential Model

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [ ]:
# Prevent GPU complete consumption
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try: 
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
    except RunTimeError as e:
        print(e)

In [13]:
model = Sequential()
# Create the embedding layer 
model.add(Embedding(MAX_FEATURES+1, 32))
# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# Final layer 
model.add(Dense(6, activation='sigmoid'))

In [14]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 512)               131584    
                                                                 
 dense_3 (Dense)             (None, 256)               131328    
                                                        

In [16]:
history = model.fit(train, epochs=10, validation_data=val)

Epoch 1/10
6981/6981 [==============================] - 1419s 202ms/step - loss: 0.0636 - val_loss: 0.0476
Epoch 2/10
6981/6981 [==============================] - 1411s 202ms/step - loss: 0.0466 - val_loss: 0.0402
Epoch 3/10
6981/6981 [==============================] - 1387s 199ms/step - loss: 0.0415 - val_loss: 0.0394
Epoch 4/10
6981/6981 [==============================] - 1437s 206ms/step - loss: 0.0387 - val_loss: 0.0344
Epoch 5/10
6981/6981 [==============================] - 1426s 204ms/step - loss: 0.0358 - val_loss: 0.0320
Epoch 6/10
6981/6981 [==============================] - 1438s 206ms/step - loss: 0.0323 - val_loss: 0.0287
Epoch 7/10
6981/6981 [==============================] - 1435s 205ms/step - loss: 0.0292 - val_loss: 0.0266
Epoch 8/10
6981/6981 [==============================] - 1439s 206ms/step - loss: 0.0272 - val_loss: 0.0240
Epoch 9/10
6981/6981 [==============================] - 1419s 203ms/step - loss: 0.0250 - val_loss: 0.0229
Epoch 10/10
6981/6981 [==============

In [18]:
from matplotlib import pyplot as plt

In [19]:
plt.figure(figsize=(8,5))
pd.DataFrame(history.history).plot()
plt.show()

NameError: name 'history' is not defined

<Figure size 576x360 with 0 Axes>

# 3. Make Predictions

In [19]:
input_text = vectorizer('You freaking suck! I am going to hit you.')

In [23]:
res = model.predict(np.expand_dims(input_text,0))

In [24]:
(res > 0.5).astype(int)

array([[1, 0, 1, 0, 1, 0]])

In [21]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [22]:
(model.predict(batch_X) > 0.5).astype(int)

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [25]:
res.shape

(1, 6)

# 4. Evaluate Model

In [26]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [27]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [28]:
for batch in test.as_numpy_iterator(): 
    # Unpack the batch 
    X_true, y_true = batch
    # Make a prediction 
    yhat = model.predict(X_true)
    
    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

In [29]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.8233095407485962, Recall:0.6706337928771973, Accuracy:0.4613841474056244


# 5. Test and Gradio

In [ ]:
!pip install gradio jinja2

In [ ]:
import tensorflow as tf
import gradio as gr

In [17]:
model.save('toxicity.h5')

In [19]:
pickle.dump(vectorizer,open('vectorizer.pkl','wb'))

InvalidArgumentError: Cannot convert a Tensor of dtype resource to a NumPy array.

In [20]:
vectorizer.save('vectorizer.h5')

AttributeError: 'TextVectorization' object has no attribute 'save'

In [21]:
model = tf.keras.models.load_model('toxicity.h5')

In [ ]:
input_str = vectorizer('hey i freaken hate you!')

In [ ]:
.

In [ ]:
res

In [24]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [25]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [ ]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

In [ ]:
interface.launch(share=True)

In [10]:
vmodel = tf.keras.models.Sequential()
vmodel.add(tf.keras.Input(shape=(1,), dtype=tf.string))
vmodel.add(vectorizer)

In [17]:
vmodel.save("vmmm", save_format="tf")

INFO:tensorflow:Assets written to: vmmm\assets


In [18]:
loaded_model = tf.keras.models.load_model("vmmm")
loaded_vectorizer = loaded_model.layers[0]

In [19]:
input_text = loaded_vectorizer('You freaking suck! I am going to hit you.')

In [22]:
res = model.predict(np.expand_dims(input_text,0))

In [23]:
(res > 0.5).astype(int)

array([[1, 0, 1, 0, 1, 0]])